In [1]:
import cv2
import numpy as np
import os
import glob
from skimage import io, transform

from sklearn import tree
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [2]:
path = './data/data/'
classname = [x for x in os.listdir(path) if os.path.isdir(path + x)]

In [3]:
def calcSURF(img):
    surf = cv2.xfeatures2d.SURF_create()
    kp, des = surf.detectAndCompute(img, None)
    return des

In [4]:
def calcFV(features, centers):
    featureVector = np.zeros((1,100))
    for i in range(0, features.shape[0]):
        fi = features[i]
        diffMat = np.tile(fi, (100, 1)) - centers
        sqSum = (diffMat**2).sum(axis = 1)
        dist = sqSum**0.5
        sortedIndices = dist.argsort()
        idx = sortedIndices[0]
        featureVector[0][idx] +=1
    return featureVector

In [5]:
def read_img(path):
    cate = [path + x for x in os.listdir(path) if os.path.isdir(path + x)]    
    for idx, folder in enumerate(cate):
        featureSet = np.float32([]).reshape(0,64)
        for im in glob.glob(folder + '/*.jpg'):
            print('reading the images:%s' % (im))
            img = io.imread(im)
            des = calcSURF(img)
            featureSet = np.append(featureSet, des, axis = 0)
        featureCount = featureSet.shape[0]
        filename = './data/svm/features/'+ classname[idx]+ '.npy'
        np.save(filename, featureSet)

In [6]:
def learnVocabulary():
    wordCount = 100
    for i in range(len(classname)):
        filename = './data/svm/features/'+ classname[i]+ '.npy'
        features = np.load(filename)
        
        print('Learning vocabulary of {}.'.format(classname[i]))
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 50, 1)  
        flags = cv2.KMEANS_RANDOM_CENTERS  
        compactness, labels, centers = cv2.kmeans(data = features, K = wordCount,bestLabels = None, criteria = criteria, attempts = 20, flags = flags)  
        
        filename = "./data/svm/vocabulary/" + classname[i] + ".npy"  
        np.save(filename, (labels, centers))  

In [7]:
def predict_response(response_true,response_predict):
    correct = (response_true==response_predict)
    return correct.mean()

In [8]:
def dataProcess():
    data = np.float32([]).reshape(0,100)
    response = np.float32([])
    imgcount = 0
    
    dicIdx = 0
    for i in range(len(classname)):
        imgpath = './data/small_data/'+ classname[i]
        labels, centers = np.load("./data/svm/vocabulary/" + classname[i] + ".npy") 
        for im in glob.glob(imgpath + '/*.jpg'):
            print('reading the images:%s' % (im))
            img = io.imread(im)
            features = calcSURF(img)
            print(features)
            featureVector = calcFV(features, centers)
            print(centers)
            print(featureVector)
            data = np.append(data, featureVector, axis = 0)
            imgcount += 1
        res = np.repeat(np.float32([dicIdx]), imgcount)
        response = np.append(response, res)
        dicIdx += 1
        imgcount = 0
    
    num_example = data.shape[0]
    arr = np.arange(num_example)
    np.random.shuffle(arr)
    data = data[arr]
    response = response[arr]
    
    ratio = 0.8
    s = np.int(num_example * ratio)
    images_train = data[:s]
    images_test = data[s:]
    response_train = response[:s]
    response_test = response[s:]
    
    return images_train, images_test, np.asarray(response_train, np.int32), np.asarray(response_test, np.int32)

In [9]:
#read_img(path = path)

In [10]:
#learnVocabulary()

In [11]:
images_train, images_test, response_train, response_test = dataProcess()

reading the images:./data/small_data/scatter plot/8.jpg
[[ 5.8761860e-05 -1.7903556e-04  5.8761860e-05 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 4.1417203e-05 -3.7032642e-05  8.0495432e-05 ... -1.4220234e-07
   1.4220234e-07  1.4220234e-07]
 [ 8.4686273e-04 -2.7047659e-03  1.7261177e-03 ... -3.0102981e-03
   3.0900887e-03  3.4391703e-03]
 ...
 [ 1.0461071e-02 -5.7743452e-03  2.1527544e-02 ...  4.2577125e-03
   1.0251508e-03  5.9528458e-03]
 [-5.3692935e-03  2.5489619e-03  8.5215187e-03 ...  1.8263387e-03
   7.1158814e-03  2.3402309e-03]
 [-3.9338889e-03 -6.7570498e-03  4.9326695e-03 ... -1.2656281e-03
   6.7632715e-03  2.3411629e-03]]
[[-3.7099732e-04 -1.1146694e-03  7.0571247e-03 ... -3.3646091e-04
   3.9076665e-03  3.4568277e-03]
 [ 2.1665131e-03 -2.2423409e-04  4.4172648e-03 ... -2.6265948e-07
   2.6210470e-02  2.8514708e-03]
 [ 1.6948706e-02 -5.2664435e-04  3.5358563e-02 ...  6.0059520e-04
   7.3828204e-03  9.4217677e-03]
 ...
 [ 1.0347165e-04  9.5493604e-05  7.6483507

[[-1.8852957e-06  3.9252125e-05  4.3213695e-05 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [-3.5779830e-03 -1.8828078e-03  4.7147679e-03 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 3.4952884e-06 -7.8196990e-07  3.4952884e-06 ... -1.9108600e-04
   4.4368403e-03  5.7462398e-03]
 ...
 [-7.4671196e-05 -2.9591157e-04  9.1782640e-05 ... -6.2739686e-03
   2.8858767e-03  2.3480067e-02]
 [ 2.4524578e-03  2.0551987e-03  3.8887837e-03 ...  1.0510731e-04
   5.1617026e-03  7.4196293e-04]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]]
[[-3.7099732e-04 -1.1146694e-03  7.0571247e-03 ... -3.3646091e-04
   3.9076665e-03  3.4568277e-03]
 [ 2.1665131e-03 -2.2423409e-04  4.4172648e-03 ... -2.6265948e-07
   2.6210470e-02  2.8514708e-03]
 [ 1.6948706e-02 -5.2664435e-04  3.5358563e-02 ...  6.0059520e-04
   7.3828204e-03  9.4217677e-03]
 ...
 [ 1.0347165e-04  9.5493604e-05  7.6483507e-03 ... -1.3603097e-04
   7.3072207e-03  6.5221679e-03]

[[ 3.6382262e-05 -8.8306238e-05  4.1667143e-05 ... -4.3219872e-05
   2.6969374e-05  4.3753789e-05]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  8.0355052e-03
   2.9540646e-03  8.0355052e-03]
 [ 4.9921236e-05 -1.2063742e-04  1.2722990e-04 ... -5.8743765e-04
   1.3566276e-02  4.7930446e-03]
 ...
 [ 7.3224882e-04  1.3531755e-03  9.8949845e-04 ...  2.7035165e-03
   3.7995491e-03  2.7198780e-03]
 [-1.5662393e-02  1.0763420e-03  1.9323282e-02 ... -3.3299695e-04
   3.6515377e-04  3.3299695e-04]
 [-5.4081305e-05  2.3373259e-05  5.4081305e-05 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]]
[[-3.7099732e-04 -1.1146694e-03  7.0571247e-03 ... -3.3646091e-04
   3.9076665e-03  3.4568277e-03]
 [ 2.1665131e-03 -2.2423409e-04  4.4172648e-03 ... -2.6265948e-07
   2.6210470e-02  2.8514708e-03]
 [ 1.6948706e-02 -5.2664435e-04  3.5358563e-02 ...  6.0059520e-04
   7.3828204e-03  9.4217677e-03]
 ...
 [ 1.0347165e-04  9.5493604e-05  7.6483507e-03 ... -1.3603097e-04
   7.3072207e-03  6.5221679e-03]

[[-6.1128857e-03 -2.4186438e-03  1.1784262e-02 ...  2.8497779e-03
   7.5732861e-03  7.3657082e-03]
 [-1.9454679e-03 -1.9149527e-04  1.9728357e-03 ... -2.5028884e-04
   1.2148942e-03  4.1593434e-04]
 [ 8.2579849e-04 -2.7234138e-03  1.7522654e-03 ... -5.2314432e-04
   8.4853172e-03  9.2020817e-04]
 ...
 [ 1.0062678e-03  2.0545428e-03  1.8933726e-03 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 1.5547236e-03 -1.1825104e-03  1.9649903e-03 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [-4.7122937e-04 -7.6994234e-05  4.7122937e-04 ... -7.3879487e-03
   2.2245968e-02  4.2231243e-02]]
[[-7.3763682e-04 -3.2765686e-04  5.0908281e-03 ...  1.8756618e-04
   5.0891815e-03  2.7326555e-03]
 [-4.1977729e-04 -2.7644850e-04  4.7002649e-03 ... -3.7652944e-04
   3.4559309e-03  2.8948307e-03]
 [-2.5321201e-03 -3.5220467e-02  5.9355758e-03 ...  6.9336621e-03
   1.3425956e-02  9.8108156e-03]
 ...
 [ 3.2367514e-04  1.7224884e-05  2.2130760e-03 ... -6.2341045e-05
   2.0656283e-03  1.8477915e-03]

[[ 5.5813356e-03 -2.7564764e-03  6.7248088e-03 ... -1.3995843e-04
   5.2125275e-04  1.3995843e-04]
 [ 3.3622934e-03 -4.3502967e-03  4.1165925e-03 ... -5.2897190e-04
   9.4371651e-05  5.2897190e-04]
 [-1.5990331e-03 -1.0707823e-03  1.6029490e-03 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 ...
 [ 5.4805493e-04  8.9554181e-03  4.7030235e-03 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [-1.3612292e-03  4.4388795e-05  6.5376181e-03 ...  6.2591491e-05
   6.2591491e-05  6.2591491e-05]
 [ 7.9684155e-03 -1.4171764e-03  1.1201836e-02 ... -4.1750083e-03
   1.3322353e-02  1.0062500e-02]]
[[-7.3763682e-04 -3.2765686e-04  5.0908281e-03 ...  1.8756618e-04
   5.0891815e-03  2.7326555e-03]
 [-4.1977729e-04 -2.7644850e-04  4.7002649e-03 ... -3.7652944e-04
   3.4559309e-03  2.8948307e-03]
 [-2.5321201e-03 -3.5220467e-02  5.9355758e-03 ...  6.9336621e-03
   1.3425956e-02  9.8108156e-03]
 ...
 [ 3.2367514e-04  1.7224884e-05  2.2130760e-03 ... -6.2341045e-05
   2.0656283e-03  1.8477915e-03]

...

[[-1.0314320e-05  7.7401186e-05  1.0314320e-05 ...  1.4668090e-05
   1.4668090e-05  1.4668090e-05]
 [ 2.0318044e-05 -5.7142457e-05  3.2857970e-05 ... -6.5053129e-05
   1.0963024e-05  6.5053129e-05]
 [-5.9255259e-03 -3.2141230e-03  8.1489785e-03 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 ...
 [-6.6278997e-04 -2.1654426e-04  6.7122251e-04 ... -1.9607980e-05
   2.0824128e-04  3.4375518e-04]
 [-7.1167275e-03  7.3836865e-03  8.4275678e-03 ...  1.2559368e-03
   6.3662382e-04  1.2559368e-03]
 [ 6.3137319e-03 -2.1255328e-03  8.1983060e-03 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]]
[[ 0.03841923 -0.00945317  0.03952678 ... -0.00011967  0.00046072
   0.001094  ]
 [-0.00043534 -0.00021096  0.00562185 ...  0.00045568  0.0051492
   0.00528658]
 [-0.00059432 -0.00123505  0.00270559 ...  0.00129315  0.06751654
   0.0056797 ]
 ...
 [ 0.02959397 -0.001529    0.03642268 ... -0.00077051  0.00423104
   0.00336509]
 [-0.00056148 -0.00086601  0.00353169 ... -0.00020847  0.00384721
   0.0

[[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [-1.6923970e-05 -5.1062274e-05  2.6083660e-05 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 4.6114085e-04 -2.3233388e-03  9.6123683e-04 ...  3.4804803e-03
   9.2606060e-03  6.7584906e-03]
 ...
 [ 1.8307754e-03 -9.0318276e-03  2.7903852e-03 ... -3.8580263e-05
   7.2523799e-05  8.0139318e-05]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [-1.4540189e-02  1.0752867e-03  1.4546985e-02 ... -1.9947491e-03
   3.3816469e-03  2.9432494e-03]]
[[ 0.03841923 -0.00945317  0.03952678 ... -0.00011967  0.00046072
   0.001094  ]
 [-0.00043534 -0.00021096  0.00562185 ...  0.00045568  0.0051492
   0.00528658]
 [-0.00059432 -0.00123505  0.00270559 ...  0.00129315  0.06751654
   0.0056797 ]
 ...
 [ 0.02959397 -0.001529    0.03642268 ... -0.00077051  0.00423104
   0.00336509]
 [-0.00056148 -0.00086601  0.00353169 ... -0.00020847  0.00384721
   0.0

[[-9.13930053e-05 -5.27423435e-05  9.13930053e-05 ... -3.57463432e-06
   7.58203532e-05  1.48916788e-05]
 [ 4.02546860e-03 -1.56471878e-03  7.50100333e-03 ... -1.96018233e-03
   7.83379283e-03  2.96989642e-03]
 [ 3.66511103e-03 -1.47911138e-03  6.81931479e-03 ...  1.71918026e-03
   1.52374711e-03  1.71918026e-03]
 ...
 [-1.26909371e-03  2.53939931e-03  1.26909371e-03 ... -3.85187828e-04
   1.68589773e-04  9.99079552e-04]
 [-3.84683604e-03 -5.87873219e-04  3.98060400e-03 ... -1.57330139e-03
   4.91356244e-04  1.57330139e-03]
 [-1.03426864e-04 -1.12275084e-05  2.08190861e-04 ...  9.61509719e-03
   9.45428014e-03  9.88020003e-03]]
[[ 0.03841923 -0.00945317  0.03952678 ... -0.00011967  0.00046072
   0.001094  ]
 [-0.00043534 -0.00021096  0.00562185 ...  0.00045568  0.0051492
   0.00528658]
 [-0.00059432 -0.00123505  0.00270559 ...  0.00129315  0.06751654
   0.0056797 ]
 ...
 [ 0.02959397 -0.001529    0.03642268 ... -0.00077051  0.00423104
   0.00336509]
 [-0.00056148 -0.00086601  0.0035316

[[ 2.3089915e-03 -2.7095864e-03  9.0146177e-03 ...  4.8376965e-05
   2.0502097e-04  6.0317314e-05]
 [ 4.4968340e-04 -4.4708643e-03  6.0517485e-03 ... -4.1672406e-03
   1.6515735e-03  4.1846987e-03]
 [-6.9014437e-05  1.5045144e-03  6.9014437e-05 ...  8.8675530e-05
   6.4514228e-05  1.4738843e-04]
 ...
 [ 3.2054351e-03  3.3762553e-03  3.5393073e-03 ... -1.6550157e-04
   1.0675255e-02  2.2124714e-04]
 [ 2.2234521e-03  8.7784947e-04  6.9344766e-03 ... -2.3155237e-04
   4.8336638e-03  3.1714269e-04]
 [-2.5757082e-04 -1.2374383e-03  6.5771240e-04 ...  7.8414585e-03
   2.2481149e-03  1.3782232e-02]]
[[ 0.03841923 -0.00945317  0.03952678 ... -0.00011967  0.00046072
   0.001094  ]
 [-0.00043534 -0.00021096  0.00562185 ...  0.00045568  0.0051492
   0.00528658]
 [-0.00059432 -0.00123505  0.00270559 ...  0.00129315  0.06751654
   0.0056797 ]
 ...
 [ 0.02959397 -0.001529    0.03642268 ... -0.00077051  0.00423104
   0.00336509]
 [-0.00056148 -0.00086601  0.00353169 ... -0.00020847  0.00384721
   0.0

[[ 3.4081459e-05 -2.0853589e-04  4.5822264e-05 ... -6.0155497e-05
   1.5867131e-03  2.5341968e-04]
 [-2.1113062e-04  1.6195269e-03  2.2174507e-04 ... -1.0419849e-06
   2.7595843e-06  1.4217923e-06]
 [ 1.8070181e-03  3.8926881e-03  4.7097667e-03 ...  1.1937658e-03
   2.0784358e-03  2.4282669e-03]
 ...
 [ 3.9722462e-04 -6.2156882e-04  3.5399385e-03 ...  1.2400791e-03
   1.4051902e-03  3.3012112e-03]
 [ 1.7618457e-03  3.9725093e-04  2.3028320e-03 ...  1.6037523e-06
   1.6037523e-06  1.6037523e-06]
 [-1.5037779e-02  1.2794324e-05  2.6804373e-02 ... -9.6461685e-05
   4.2012113e-04  3.9771141e-04]]
[[ 0.03841923 -0.00945317  0.03952678 ... -0.00011967  0.00046072
   0.001094  ]
 [-0.00043534 -0.00021096  0.00562185 ...  0.00045568  0.0051492
   0.00528658]
 [-0.00059432 -0.00123505  0.00270559 ...  0.00129315  0.06751654
   0.0056797 ]
 ...
 [ 0.02959397 -0.001529    0.03642268 ... -0.00077051  0.00423104
   0.00336509]
 [-0.00056148 -0.00086601  0.00353169 ... -0.00020847  0.00384721
   0.0

In [12]:
ESTIMATORS = {
    "Logistic regression": LogisticRegression(),
    "SVC": SVC(),
    "Random Forest": RandomForestClassifier(),
    "Decision Tree": tree.DecisionTreeClassifier(),
}

In [13]:
images_test_predict = dict()
for name, estimator in ESTIMATORS.items():
    print("estimator: {} fit start".format(name))
    estimator.fit(images_train, response_train)
    print("estimator: {} fit complete".format(name))
    images_test_predict[name] = estimator.predict(images_test)
    print("predict_complete: {}".format(name))

estimator: Logistic regression fit start
estimator: Logistic regression fit complete
predict_complete: Logistic regression
estimator: SVC fit start
estimator: SVC fit complete
predict_complete: SVC
estimator: Random Forest fit start
estimator: Random Forest fit complete
predict_complete: Random Forest
estimator: Decision Tree fit start
estimator: Decision Tree fit complete
predict_complete: Decision Tree


In [14]:
for name, estimator in ESTIMATORS.items():
    predict = predict_response(response_test,images_test_predict[name])
    print("{0}: {1:.1%}".format(name,predict))

Logistic regression: 94.3%
SVC: 25.7%
Random Forest: 97.1%
Decision Tree: 82.9%


In [15]:
images_test_predict["Logistic regression"]

array([4, 6, 6, 3, 3, 1, 3, 7, 6, 6, 3, 6, 1, 7, 1, 7, 7, 3, 3, 6, 1, 4,
       7, 7, 7, 6, 6, 7, 1, 0, 2, 7, 6, 6, 7], dtype=int32)

In [16]:
response_test

array([4, 6, 6, 3, 3, 1, 3, 7, 6, 6, 3, 5, 3, 7, 1, 7, 7, 3, 3, 6, 1, 4,
       7, 7, 7, 6, 6, 7, 1, 0, 2, 7, 6, 6, 7], dtype=int32)